In [34]:
options(warn = -1)

In [35]:
library(lubridate)
library(rgdal)
library(stringr)


Attaching package: 'lubridate'


The following objects are masked from 'package:base':

    date, intersect, setdiff, union


Loading required package: sp

rgdal: version: 1.5-23, (SVN revision 1121)
Geospatial Data Abstraction Library extensions to R successfully loaded
Loaded GDAL runtime: GDAL 3.2.1, released 2020/12/29
Path to GDAL shared files: C:/Users/clid1852/Documents/R/win-library/4.0/rgdal/gdal
GDAL binary built with GEOS: TRUE 
Loaded PROJ runtime: Rel. 7.2.1, January 1st, 2021, [PJ_VERSION: 721]
Path to PROJ shared files: C:/Users/clid1852/Documents/R/win-library/4.0/rgdal/proj
PROJ CDN enabled: TRUE
Linking to sp version:1.4-5
To mute warnings of possible GDAL/OSR exportToProj4() degradation,
use options("rgdal_show_exportToProj4_warnings"="none") before loading rgdal.
Overwritten PROJ_LIB was C:/Users/clid1852/Documents/R/win-library/4.0/rgdal/proj



In [1]:
path <- "T:/DCProjects/StoryMap/BikeCounting/BikeShare/Data/Output/review"

In [10]:
files <- list.files(path)[grep("2021", list.files(path))]

In [16]:
selected_vars <- c('User.ID', 'Route.ID', 'Start.Hub', 
                   'Start.Latitude', 'Start.Longitude',
                   'Start.Date', 'Start.Time', 
                   'End.Hub', 'End.Latitude', 'End.Longitude',
                   'End.Date', 'End.Time', 'Bike.ID', 'Bike.Name',
                   'Distance..Miles.', 'Duration')

In [17]:
for(file in files){
    #print(file)
    if(file == files[1]){
        df <- read.csv(paste0(path, "/", file))
        df <- df[, selected_vars]
    }else{
        ndf <- read.csv(paste0(path, "/", file))
        ndf <- ndf[, selected_vars]
        df <- rbind(df, ndf)
    }
}

In [9]:
toMinutes <- function(x){
  h <- as.numeric(strsplit(x, ":")[[1]][1])
  m <- as.numeric(strsplit(x, ":")[[1]][2])
  s <- as.numeric(strsplit(x, ":")[[1]][3])
  
  res <- h*60 + m + s/60
  
  return(res)
}

In [18]:
df$Minutes <- unlist(lapply(df$Duration, function(x) toMinutes(x)))

In [23]:
df$Path.ID = paste(df$Start.Hub, "-", df$End.Hub)

In [29]:
organize_data <- function(df){
  #trips <- read.csv(paste0(inpath, "/", file))
  org <- df[,c('Route.ID', 'Bike.ID', 'User.ID', 
                  'Start.Hub', 'Start.Latitude', 'Start.Longitude',
                  'Start.Date', 'Start.Time', 'Path.ID',
                  'Distance..Miles.', 'Minutes')]
  names(org) <- c("RouteID", "BikeID", 'UserID',
                  "Location", "Latitude", "Longitude",
                  "Date", "Time", 'PathID',
                  'Distance', 'Minutes')
  org$OrgDst <- rep("Origin", dim(org)[1])
  dst <- df[,c('Route.ID', 'Bike.ID', 'User.ID', 'End.Hub',
                  'End.Latitude', 'End.Longitude',
                  'End.Date','End.Time', 'Path.ID',
                  'Distance..Miles.', 'Minutes')]
  names(dst) <- c("RouteID", "BikeID", 'UserID',
                  "Location", "Latitude", "Longitude", 
                  "Date", "Time", 'PathID',
                  'Distance', 'Minutes')
  dst$OrgDst <- rep("Destination", dim(dst)[1])
  ndf <- rbind(org, dst)
  return(ndf)
}

In [26]:
# focus on CLMPO
mdf <- df[(df$Start.Latitude >= 43.97865 & df$Start.Latitude <= 44.16123) & 
           (df$Start.Longitude >= -123.2321 & df$Start.Longitude <= -122.8281) & 
           (df$End.Latitude >= 43.97865 & df$End.Latitude <= 44.16123) &
           (df$End.Longitude >= -123.2321 & df$End.Longitude <= -122.8281), ]

mdf <- mdf[!(mdf$Start.Longitude == mdf$End.Longitude & mdf$Start.Latitude == mdf$End.Latitude),]

In [25]:
names(df)

[1] "User.ID"          "Route.ID"         "Start.Hub"        "Start.Latitude"  
 [5] "Start.Longitude"  "Start.Date"       "Start.Time"       "End.Hub"         
 [9] "End.Latitude"     "End.Longitude"    "End.Date"         "End.Time"        
[13] "Bike.ID"          "Bike.Name"        "Distance..Miles." "Duration"        
[17] "Minutes"          "Path.ID"

In [30]:
ndf <- organize_data(mdf)

In [39]:
head(ndf$Date)

[1] "2021-01-01" "2021-01-01" "2021-01-01" "2021-01-01" "2021-01-01"
[6] "2021-01-01"

In [40]:
ndf$Date <- as.Date(ndf$Date, "%Y-%m-%d")

In [54]:
ndf$Weekday <- as.character(wday(ndf$Date, label=TRUE, abbr=FALSE))

In [44]:
ndf$Month <- months(ndf$Date)

In [45]:
ndf$Season <- ifelse(ndf$Month %in% c("December", "January", "February"), "Winter",
                    ifelse(ndf$Month %in% c("March", "April", "May"), "Spring",
                          ifelse(ndf$Month %in% c("June", "July", "August"), "Summer", "Fall")))

In [46]:
ndf$SeasonOrder <- ifelse(ndf$Season == "Spring", 1, 
                              ifelse(ndf$Season == "Summer", 2, 
                                     ifelse(ndf$Season == "Fall", 3, 4)))

In [47]:
ndf$WeekdayOrder <- ifelse(ndf$Weekday == "Monday", 1, 
                              ifelse(ndf$Weekday == "Tuesday", 2, 
                                     ifelse(ndf$Weekday == "Wednesday", 3, 
                                     ifelse(ndf$Weekday == "Thursday", 4, 
                                            ifelse(ndf$Weekday == "Friday", 5, 
                                                ifelse(ndf$Weekday == "Saturday", 6, 7))))))

In [48]:
outpath <- "T:/DCProjects/StoryMap/BikeCounting/BikeShare/Output"

In [51]:
dim(ndf)

[1] 179122     17

In [50]:
write.csv(ndf, paste0(outpath, "/Bike_Share_Trips.csv"), row.names = FALSE)

In [58]:
head(ndf)

,RouteID,BikeID,UserID,Location,Latitude,Longitude,Date,Time,PathID,Distance,Minutes,OrgDst,Weekday,Month,Season,SeasonOrder,WeekdayOrder
,<int>,<int>,<int>,<chr>,<dbl>,<dbl>,<date>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>
1,13713102,19521,783398,Monroe St & Blair Blvd,44.05141,-123.1049,2021-01-01,01:11,Monroe St & Blair Blvd - 15th & Ferry,1.55,11.000000,Origin,Friday,January,Winter,4,5
2,13713106,19584,1510848,PeaceHealth University District,44.04736,-123.0822,2021-01-01,01:52,PeaceHealth University District - PeaceHealth University District,1.17,20.550000,Origin,Friday,January,Winter,4,5
3,13713112,19737,911293,12th & Willamette,44.04608,-123.0929,2021-01-01,03:16,"12th & Willamette - 17th & Pearl, East side",0.49,4.916667,Origin,Friday,January,Winter,4,5
4,13713256,19737,766725,"17th & Pearl, East side",44.04109,-123.0900,2021-01-01,09:36,"17th & Pearl, East side - 17th & Pearl, East side",2.29,23.100000,Origin,Friday,January,Winter,4,5
5,13713355,19761,1726219,15th & Ferry,44.04355,-123.0852,2021-01-01,11:09,15th & Ferry - 15th & Ferry,0.01,1.100000,Origin,Friday,January,Winter,4,5
6,13713360,19711,1174607,Valley River,44.06523,-123.1037,2021-01-01,11:13,Valley River - Valley River,7.60,36.500000,Origin,Friday,January,Winter,4,5


In [36]:
MPOBound <- readOGR(dsn = "V:/Data/Transportation", layer="MPO_Bound")

OGR data source with driver: ESRI Shapefile 
Source: "V:\Data\Transportation", layer: "MPO_Bound"
with 1 features
It has 3 fields


In [32]:
df2spdf <- function(df, lon_col_name, lat_col_name, trans = TRUE){
  lonlat <- sp::CRS("+proj=longlat +datum=WGS84 +ellps=WGS84 +towgs84=0,0,0")
  lon_col_no <- which(names(df)==lon_col_name)
  lat_col_no <- which(names(df)==lat_col_name)
  xy <- data.frame(df[,c(lon_col_no,lat_col_no)])
  coordinates(xy) <- c(lon_col_name, lat_col_name)
  proj4string(xy) <- lonlat
  spdf <- sp::SpatialPointsDataFrame(coords = xy, data = df)
  if(trans){
    spdf <- spTransform(spdf, CRS(proj4string(MPOBound)))
  }
  return(spdf)
}

In [55]:
spdf <- df2spdf(ndf, 'Longitude', 'Latitude')

In [57]:
writeOGR(spdf, dsn=outpath, layer="Bike_Share_Trips", driver="ESRI Shapefile", overwrite_layer=TRUE)

In [59]:
names(ndf)

[1] "RouteID"      "BikeID"       "UserID"       "Location"     "Latitude"    
 [6] "Longitude"    "Date"         "Time"         "PathID"       "Distance"    
[11] "Minutes"      "OrgDst"       "Weekday"      "Month"        "Season"      
[16] "SeasonOrder"  "WeekdayOrder"

In [60]:
datedf <- unique(ndf[,c("Date", "Month", "Season", "Weekday", "SeasonOrder", "WeekdayOrder")])

In [160]:
datedf$MonthOrder <- ifelse(datedf$Month == "January", 1, 
                           ifelse(datedf$Month == "February", 2, 
                               ifelse(datedf$Month == "March", 3, 
                                   ifelse(datedf$Month == "April", 4, 
                                       ifelse(datedf$Month == "May", 5, 
                                           ifelse(datedf$Month == "June", 6, 
                                               ifelse(datedf$Month == "July", 7, 
                                                   ifelse(datedf$Month == "August", 8, 
                                                       ifelse(datedf$Month == "September", 9, 
                                                           ifelse(datedf$Month == "October", 10, 
                                                               ifelse(datedf$Month == "November", 11, 12)))))))))))

In [62]:
dim(ndf)

[1] 179122     17

In [61]:
locdf <- unique(ndf[,c("Location", "Latitude", "Longitude", "OrgDst")])

In [63]:
dim(locdf)

[1] 148659      4

In [92]:
stations <- read.csv("T:/DCProjects/GitHub/BikeCounting/BikeMap/BikeShareStations.csv")

In [70]:
head(stations)

,station.id,name,lon,lat
,<chr>,<chr>,<dbl>,<dbl>
1,hub_3828,Eugene Family YMCA,-123.0835,44.03754
2,hub_3829,Monroe St & Blair Blvd,-123.1051,44.05147
3,hub_3830,10th & Olive SE,-123.0941,44.04866
4,hub_3831,16th & Willamette,-123.0929,44.04219
5,hub_3832,13th & Pearl @ Pearl side,-123.0897,44.04526
6,hub_3840,PeaceHealth University District,-123.0816,44.04757


In [93]:
length(unique(stations$name))

[1] 60

In [94]:
stations$name <- str_replace(stations$name, " @", ",")

In [100]:
stations[grep("RiverBend", stations$name),]

,station.id,name,lon,lat
,<chr>,<chr>,<dbl>,<dbl>
36,hub_3875,PeaceHealth RiverBend,-123.0272,44.08182
42,hub_5512,RiverBend Annex,-123.0301,44.08776


In [101]:
hub_df <- ndf[ndf$Location %in% stations$name,]

In [102]:
dim(hub_df)

[1] 147082     17

In [118]:
unique(hub_df$OrgDst)

[1] "Origin"      "Destination"

In [161]:
aggdata <- aggregate(x=list(NoTrips = hub_df$RouteID, NoUsers = hub_df$UserID), 
                     by=list(Date = hub_df$Date, Location = hub_df$Location), 
                     FUN=function(x) length(x))

In [134]:
get_aggdata <- function(df=hub_df, OrgDst="Origin"){
    df <- df[df$OrgDst == OrgDst,]
    aggdata <- aggregate(x=list(NoTrips = df$RouteID, NoUsers = df$UserID), 
                     by=list(Date = df$Date, Location = df$Location), 
                     FUN=function(x) length(x))

    if(OrgDst=="Origin"){
        names(aggdata)[3:4] <- c("OrgTrips", "OrgUsers")
    }else{
        names(aggdata)[3:4] <- c("DstTrips", "DstUsers")
    }
    
    return(aggdata)    
}

In [162]:
aggdata <- merge(aggdata, get_aggdata(df=hub_df, OrgDst="Origin"), by=c("Location", "Date"))

In [163]:
aggdata <- merge(aggdata, get_aggdata(df=hub_df, OrgDst="Destination"), by=c("Location", "Date"))

In [164]:
names(stations) <- c("StationID", "Location", "Longitude", "Latitude")

In [165]:
aggdata <- merge(aggdata, stations, by="Location")

In [166]:
aggdata <- merge(aggdata, datedf, by="Date")

In [167]:
dim(aggdata)

[1] 11823    17

In [168]:
head(aggdata)

,Date,Location,NoTrips,NoUsers,OrgTrips,OrgUsers,DstTrips,DstUsers,StationID,Longitude,Latitude,Month,Season,Weekday,SeasonOrder,WeekdayOrder,MonthOrder
,<date>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
1,2021-01-01,"5th & Pearl, NW corner",12,12,6,6,6,6,hub_3842,-123.0898,44.05455,January,Winter,Friday,4,5,1
2,2021-01-01,Amazon Pool,2,2,1,1,1,1,hub_5407,-123.0818,44.03022,January,Winter,Friday,4,5,1
3,2021-01-01,1600 Millrace Drive,2,2,1,1,1,1,hub_3862,-123.0687,44.04828,January,Winter,Friday,4,5,1
4,2021-01-01,13th & Kincaid,2,2,1,1,1,1,hub_3860,-123.0785,44.04584,January,Winter,Friday,4,5,1
5,2021-01-01,Valley River,8,8,4,4,4,4,hub_5415,-123.1037,44.06526,January,Winter,Friday,4,5,1
6,2021-01-01,12th & Willamette,5,5,3,3,2,2,hub_3844,-123.0928,44.04653,January,Winter,Friday,4,5,1


In [169]:
names(aggdata)

[1] "Date"         "Location"     "NoTrips"      "NoUsers"      "OrgTrips"    
 [6] "OrgUsers"     "DstTrips"     "DstUsers"     "StationID"    "Longitude"   
[11] "Latitude"     "Month"        "Season"       "Weekday"      "SeasonOrder" 
[16] "WeekdayOrder" "MonthOrder"

In [170]:
write.csv(aggdata, paste0(outpath, "/Daily_Bike_Share_Trips.csv"), row.names = FALSE)

In [178]:
sumdf <- aggregate(x=list(NoTrips = aggdata$NoTrips, 
                          NoUsers = aggdata$NoUsers, 
                          OrgTrips = aggdata$OrgTrips, 
                          OrgUsers = aggdata$OrgUsers, 
                          DstTrips = aggdata$DstTrips, 
                          DstUsers = aggdata$DstUsers), 
                   by=list(Location = aggdata$Location), 
                   FUN=sum)

In [179]:
avgdf <- aggregate(x=list(AvgNoTrips = aggdata$NoTrips, 
                          AvgNoUsers = aggdata$NoUsers), 
                   by=list(Location = aggdata$Location), 
                   FUN=mean)

In [180]:
sumdf$PctTrips <- (sumdf$OrgTrips / sumdf$NoTrips) * 100

In [181]:
sumdf$PctUsers <- (sumdf$OrgUsers / sumdf$NoUsers) * 100

In [182]:
sumdf <- merge(sumdf, stations, by="Location")

In [183]:
sum_avg_df <- merge(sumdf, avgdf, by="Location")

In [184]:
write.csv(sum_avg_df, paste0(outpath, "/Sum_Bike_Share_Trips.csv"), row.names = FALSE)

In [171]:
agg_spdf <- df2spdf(aggdata, 'Longitude', 'Latitude')

In [172]:
writeOGR(agg_spdf, dsn=outpath, layer="Daily_Bike_Share_Trips", driver="ESRI Shapefile", overwrite_layer=TRUE)

In [185]:
sum_avg_spdf <- df2spdf(sum_avg_df, 'Longitude', 'Latitude')

In [188]:
names(sum_avg_spdf)

[1] "Location"   "NoTrips"    "NoUsers"    "OrgTrips"   "OrgUsers"  
 [6] "DstTrips"   "DstUsers"   "PctTrips"   "PctUsers"   "StationID" 
[11] "Longitude"  "Latitude"   "AvgNoTrips" "AvgNoUsers"

In [190]:
writeOGR(sum_avg_spdf, dsn=outpath, layer="Sum_Bike_Share_Trips", driver="ESRI Shapefile", overwrite_layer=TRUE)